# [실습] LangGraph의 다양한 State 활용하기    


이전 실습에서는 하나의 클래스에 문자열, 정수와 같은 값을 정의하고, 이를 모든 노드가 공유하도록 구성했는데요.   

이번 실습에서는 State를 보다 복잡하게 만들어 보겠습니다.   





In [ ]:
!pip install langgraph langchain langchain_google_genai langchain_community

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

## 1. 구조화된 출력 State에 적용하기

LangChain의 `llm.with_structured_output`을 사용하면, 구조화된 출력을 만들 수 있습니다.   
Pydantic을 이용해, 예시 데이터의 구조를 만들어 보겠습니다.

In [ ]:
from pydantic import BaseModel, Field

# 프롬프트 자동 생성을 위한 요소 저장
class Objective(BaseModel):
    instruction: str = Field(description='프롬프트의 지시 사항을 명확히 재구성')
    output_format: str = Field(description='출력 포맷에 대한 설명')
    examples: str = Field(description='예시 출력(1개)')
    notes: str = Field(description='작업 과정에서 중요한 내용을 4개의 개조식 문장으로 구성')

    @property #
    def as_str(self) -> str:
        return '\n\n'.join([f'## {key}\n {value}' for key, value in self])


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ('system', '아래의 작업을 보다 자세하게 요청하는 시스템 프롬프트를 구성하고자 합니다. 주어진 포맷에 적절하게 작성하세요.'),
    ('user', '{instruction}')

])

chain = prompt | llm.with_structured_output(Objective)

result = chain.invoke("집에서 쉽게 구할 수 있는 재료로 재미있는 장난감 만들기")

result

In [ ]:
print(result.as_str)

위에서 만든 Objective Class는 State의 단일 값으로도 저장할 수 있습니다.

In [ ]:
from typing import TypedDict

class State(TypedDict):
    instruction : str
    prompt_materials : Objective # Objective Class를 하나의 값에 저장
    full_prompt : str
    result : str

In [ ]:
def get_prompt_materials(State):
    prompt = ChatPromptTemplate([
        ('system', '아래의 작업을 보다 자세하게 세분화하고자 합니다. 주어진 포맷에 적절하게 작성하세요.'),
        ('user', '{instruction}')

    ])

    chain = prompt | llm.with_structured_output(Objective)

    result = chain.invoke({'instruction':State['instruction']})
    return {'prompt_materials' : result}


In [ ]:
from langchain_core.output_parsers import StrOutputParser

def generate_prompt(State):
    prompt = ChatPromptTemplate([
        ('system', '''당신은 체계적이고 정확한 프롬프트 엔지니어입니다. 아래의 포인트를 바탕으로, LLM에 입력할  시스템 프롬프트를 작성하세요.
{points}'''),
        ('user', '{instruction}')
    ])

    chain = prompt | llm | StrOutputParser()

    result = chain.invoke({'instruction': State['instruction'], 'points': State['prompt_materials'].as_str})
    return {'full_prompt' : result}


In [ ]:
def generate(State):
    return {'result' : llm.invoke(State['full_prompt']).content}

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


# 그래프 구성
builder = StateGraph(State)

builder.add_node("get_prompt_materials", get_prompt_materials)
builder.add_node("generate_prompt", generate_prompt)
builder.add_node("generate", generate)

builder.add_edge(START, "get_prompt_materials")
builder.add_edge("get_prompt_materials", "generate_prompt")
builder.add_edge("generate_prompt", "generate")

builder.add_edge("generate", END)


In [ ]:
graph = builder.compile()
graph

In [ ]:
import pprint

# Streaming 참고
# https://langchain-ai.github.io/langgraph/concepts/streaming/#streaming-graph-outputs-stream-and-astream

for data in graph.stream({'instruction': '''영화 '마이너리티 리포트'와 AI 윤리의 연관성에 대한 리포트 쓰기'''},
                         stream_mode='values'):
    pprint.pprint(data)
    print('----')

In [ ]:
data

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(data['result'])


이와 같이 구조화된 출력을 연결하여, 그래프의 중간, 혹은 최종 출력물을 구성할 수 있습니다.

## 2. Message 포맷의 State 사용하기

State의 저장값으로 Message를 바로 사용하기도 합니다.   
이 경우, Context에 메시지를 계속 추가하거나, 별도의 로직을 만들어 메시지 정보를 전달합니다.

`typing`의 `Annotated`로 공간을 지정한 후, 뒷부분에 결합 로직을 포함합니다.   
이를 리듀서(Reducer)라고 부르는데, 메시지의 경우 아래와 같이 포함하면 됩니다.

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# add_messages: 메시지를 계속 뒤에 추가하는 방식
# 기존 메시지를 수정하거나, 삭제하는 것도 가능합니다.

class State(TypedDict):
    context : Annotated[list, add_messages]

이번에는 메시지를 주고받는 형태를 구성해 보겠습니다.   

In [ ]:
def talk(State):
    return {'context': AIMessage(content='AI 메시지 2')}


builder = StateGraph(State)
builder.add_node('talk',talk)
builder.add_edge(START, 'talk')
builder.add_edge('talk', END)





In [ ]:
graph = builder.compile()
graph

In [ ]:
messages = [
    SystemMessage(content='시스템 메시지 1'),
    HumanMessage(content='유저 메시지 1'),
    AIMessage(content='AI 메시지 1'),
    HumanMessage(content='유저 메시지 2'),
]

response = graph.invoke({'context': messages})
response

전체 Context를 모두 저장하는 위와 같은 방식도 가능하지만,   
`RemoveMessage`를 사용하여 메시지를 제거할 수도 있습니다.

In [ ]:
from langchain_core.messages import RemoveMessage
def delete_message(State):
    # 첫번째,두번째 메시지 삭제
    messages = State['context']
    return {"context": [RemoveMessage(id = messages[i].id) for i in range(1,3)]}


In [ ]:
builder = StateGraph(State)

builder.add_node('talk',talk)
builder.add_node('delete_message',delete_message)

builder.add_edge(START, 'talk')
builder.add_edge('talk', 'delete_message')
builder.add_edge('delete_message', END)

In [ ]:
graph = builder.compile()
graph

In [ ]:
messages

In [ ]:
# 유저 메시지 1, AI 메시지 1 삭제

graph.invoke({'context': messages})

위 방식으로 긴 컨텍스트를 저장할 때 일부만을 저장하거나, 앞 부분의 컨텍스트를 수정하여 저장할 수 있습니다.   
반복 기능을 추가한다면, 긴 컨텍스트의 대화도 효과적으로 만들 수 있습니다.